Greggs Vs Pret

In [1]:
import overpy #library allows you to query OpenStreetMap data using the Overpass API
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import seaborn as sns
import folium

In [2]:
api = overpy.Overpass()

In [3]:
P = api.query("""
    area["wikidata" = "Q145"]->.place;   
    node["brand:wikidata"="Q3403981"](area.place);
    way["brand:wikidata"="Q3403981"](area.place);
    (._;>;);
    out body;
    """)

In [4]:
P.nodes

[<overpy.Node id=249801499 lat=52.5850794 lon=-2.1257925>,
 <overpy.Node id=309677006 lat=55.8566374 lon=-4.0208524>,
 <overpy.Node id=309677007 lat=55.8564692 lon=-4.0208080>,
 <overpy.Node id=309677008 lat=55.8564920 lon=-4.0205338>,
 <overpy.Node id=309677009 lat=55.8566602 lon=-4.0205782>,
 <overpy.Node id=311305889 lat=51.8185129 lon=1.1669311>,
 <overpy.Node id=338507382 lat=51.5165185 lon=-0.1192080>,
 <overpy.Node id=354891376 lat=52.4433360 lon=-1.8339229>,
 <overpy.Node id=354891381 lat=52.4432708 lon=-1.8340320>,
 <overpy.Node id=357751934 lat=53.7957335 lon=-1.5444103>,
 <overpy.Node id=362323834 lat=51.5505537 lon=-0.1092759>,
 <overpy.Node id=362323836 lat=51.5506214 lon=-0.1091792>,
 <overpy.Node id=362323838 lat=51.5505173 lon=-0.1092349>,
 <overpy.Node id=362323841 lat=51.5505812 lon=-0.1091161>,
 <overpy.Node id=394822512 lat=53.5526309 lon=-1.4815682>,
 <overpy.Node id=409759996 lat=52.4674706 lon=-1.7213872>,
 <overpy.Node id=409759997 lat=52.4676243 lon=-1.7215218>

In [5]:
from shapely.geometry import Point

# creating a geodataframe using the POI data
points_array = [ Point(x.lon, x.lat) for x in P.nodes]
points_series = gpd.GeoSeries(points_array)

monuments_array = [ { 
    "name": x.tags.get("name:city", ""),
} for x in P.nodes]

sc_gdf = gpd.GeoDataFrame(monuments_array, geometry=points_series, crs=4326)

sc_gdf

,name,geometry
0,,POINT (-2.12579 52.58508)
1,,POINT (-4.02085 55.85664)
2,,POINT (-4.02081 55.85647)
3,,POINT (-4.02053 55.85649)
4,,POINT (-4.02058 55.85666)
...,...,...
2276,,POINT (-1.72068 53.78456)
2277,,POINT (-0.76381 51.24873)
2278,,POINT (-0.76414 51.24871)
2279,,POINT (-0.76412 51.24867)


In [6]:
# adding lat and lon columns based on the values in the geometry column
sc_gdf['lat'] = sc_gdf['geometry'].y
sc_gdf['lon'] = sc_gdf['geometry'].x

In [7]:
# creating a folium map 

m = folium.Map(
    location=[51.5074, -0.1272], # start location as lat and lon
    tiles="CartoDB dark_matter", # adding a dark basemap
    zoom_start=12, # level of zoom
    prefer_canvas=True, # useful for changing the base map
)

In [8]:
# iterate through every row of the dataframe using the iterrows() function
for index, val in sc_gdf.iterrows():
    folium.CircleMarker(
        location=[val["lat"], val["lon"]],
        # styling the circles with different parameters 
        radius=5,
        popup= val["name"],
        color='#3186cc',
        fill=True,
        fill_color='#3186cc'
    # adding all circles to the map
    ).add_to(m)


m

---
---
---

In [7]:
import os
import configparser

import geopandas as gpd
from shapely.geometry import Point, Polygon

import requests
import json
import pandas as pd

import matplotlib.pyplot as plt

#for legends
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
import math
from matplotlib.lines import Line2D

In [ ]:
P = api.query("""
    area["wikidata" = "Q145"]->.place;   
    node["brand:wikidata"="Q3403981"](area.place);
    way["brand:wikidata"="Q3403981"](area.place);
    (._;>;);
    out body;
    """)

In [30]:
# get Greggs from OSM using Overpass API
overpass_url = 'http://overpass-api.de/api/interpreter'
overpass_query = '''
[out:json];
area[name='Greater London'][admin_level=5]->.searchArea;

(way["brand:wikidata"="Q3403981"](area.searchArea);  
node["brand:wikidata"="Q3403981"](area.searchArea);  
rel["brand:wikidata"="Q3403981"](area.searchArea); 
);
out center;

'''
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

In [31]:
# convert json into dataframe
df_dict = {'id':[], 'type': [], 'lat':[], 'lon': [],'name':[]}

for element in data['elements']:
    df_dict['id'].append(element['id'])
    df_dict['type'].append(element['type'])
    
    for tag in list(df_dict.keys())[4:]:
        try:
            df_dict[tag].append(element['tags'][tag])
        except KeyError:
            df_dict[tag].append('') 
    if element['type']=='node':
        df_dict['lat'].append(element['lat'])
        df_dict['lon'].append(element['lon'])
    else:
        df_dict['lat'].append(element['center']['lat'])
        df_dict['lon'].append(element['center']['lon'])
        
df = pd.DataFrame.from_dict(df_dict)

# convert into geodataframe
geometry = [Point(xy) for xy in zip(df['lon'].astype(float), df['lat'].astype(float))]
greggs_gdf = gpd.GeoDataFrame(df, crs = 'EPSG:4326', geometry = geometry)

In [26]:
# get Prets from OSM using Overpass API
overpass_url = 'http://overpass-api.de/api/interpreter'
overpass_query = '''
[out:json];
area[name='Greater London'][admin_level=5]->.searchArea;

(way["brand:wikidata"="Q2109109"](area.searchArea);  
node["brand:wikidata"="Q2109109"](area.searchArea);  
rel["brand:wikidata"="Q2109109"](area.searchArea); 
);
out center;

'''
response = requests.get(overpass_url, params={'data': overpass_query})
pret_data = response.json()

In [33]:
# convert json into dataframe
df_dict = {'id':[], 'type': [], 'lat':[], 'lon': [],'name':[]}

for element in pret_data['elements']:
    df_dict['id'].append(element['id'])
    df_dict['type'].append(element['type'])
    
    for tag in list(df_dict.keys())[4:]:
        try:
            df_dict[tag].append(element['tags'][tag])
        except KeyError:
            df_dict[tag].append('') 
    if element['type']=='node':
        df_dict['lat'].append(element['lat'])
        df_dict['lon'].append(element['lon'])
    else:
        df_dict['lat'].append(element['center']['lat'])
        df_dict['lon'].append(element['center']['lon'])
        
df = pd.DataFrame.from_dict(df_dict)

# convert into geodataframe
geometry = [Point(xy) for xy in zip(df['lon'].astype(float), df['lat'].astype(float))]
pret_gdf = gpd.GeoDataFrame(df, crs = 'EPSG:4326', geometry = geometry)

In [34]:
#append the greggs and pret dataframe
appended_df = greggs_gdf.append(pret_gdf, ignore_index=True)


C:\Users\EMoses\AppData\Local\Temp\ipykernel_11640\3652567341.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  appended_df = greggs_gdf.append(pret_gdf, ignore_index=True)


In [46]:
appended_df

,id,type,lat,lon,name,geometry
0,140697749,node,51.498230,-0.135216,Greggs,POINT (-0.13522 51.49823)
1,414028109,node,51.450657,0.056367,Greggs,POINT (0.05637 51.45066)
2,420738683,node,51.403678,0.014947,Greggs,POINT (0.01495 51.40368)
3,420783204,node,51.400718,0.017109,Greggs,POINT (0.01711 51.40072)
4,601053032,node,51.509795,-0.123363,Greggs,POINT (-0.12336 51.50979)
...,...,...,...,...,...,...
403,958536904,way,51.516007,-0.084113,Pret A Manger,POINT (-0.08411 51.51601)
404,988767809,way,51.513857,-0.122739,Pret A Manger,POINT (-0.12274 51.51386)
405,997224975,way,51.494448,-0.212520,Pret A Manger,POINT (-0.21252 51.49445)
406,1002485835,way,51.493027,-0.225577,Pret A Manger,POINT (-0.22558 51.49303)


In [50]:
# creating a folium map 

m = folium.Map(
    location=[51.5074, -0.1272], # start location as lat and lon
    tiles="CartoDB dark_matter", # adding a dark basemap
    zoom_start=12, # level of zoom
    prefer_canvas=True, # useful for changing the base map
)

# Create a dictionary to map names to colors
color_dict = {
    "Pret A Manger": '#964B00',
    "Greggs": '#3186cc',
}

# Iterate through every row of the GeoDataFrame using the iterrows() function
for index, val in appended_df.iterrows():
    # Set color based on the value in the 'name' column, default to '#3186cc'
    fill_color = color_dict.get(val["name"])

    folium.CircleMarker(
        location=[val["geometry"].y, val["geometry"].x],  # Extracting lat and lon from Point geometry
        radius=5,
        popup=val["name"],
        color=fill_color,  # Outline color
        fill=True,
        fill_color=fill_color  # Fill color based on the 'name' column
    ).add_to(m)


m
